In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from tensorflow import keras
import shap

from config import DATA_DIR
from preprocessing import _downcast_dtypes, _load_userdf
from utils import _get_feature

import warnings
warnings.simplefilter("ignore")

## Loading train and test sets

In [ ]:
x_train = np.load("./generated-datasets/happy/happy_8H_7days_x_train.npy")
y_train = np.load("./generated-datasets/happy/happy_8H_7days_y_train.npy")
x_test = np.load("./generated-datasets/happy/happy_8H_7days_x_test.npy")
y_test = np.load("./generated-datasets/happy/happy_8H_7days_y_test.npy")

## Loading final RNN 

In [ ]:
# import keras for model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import Bidirectional

In [ ]:
stress_lstm = keras.models.load_model("./model_logs/best_stress_LSTM")

In [ ]:
#happy_lstm = keras.models.load_model("./model_logs/best_happy_LSTM")

# SHAP

## Plot Shap sequences

In [ ]:
def plot_seq(seq_matrix):
    rows = 2
    cols = 1

    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(9, 9))

    shap_matrix = np.mean(seq_matrix, axis=1).reshape((3, 7))
    ax[0].imshow(shap_matrix, cmap=plt.cm.get_cmap("RdBu").reversed())
    ax[0].set_xticks(np.arange(7))
    ax[0].set_xticklabels(["Mon", "Tues", "Wed", "Thu", "Fri", "Sat", "Sun"])
    ax[0].set_yticks(np.arange(3))
    ax[0].set_yticklabels(["0-8", "8-16", "16-24"])
    
    ax[1].plot(np.mean(seq_matrix, axis=1), marker="o")
    ax[1].margins(x=0)
    ax[1].set_xticks(np.arange(0, 21+1, 3), minor=False)
    ax[1].set_xticks(np.arange(0, 21+1, 1), minor=True)
    ax[1].set_yticks([-0.002, 0, 0.002], minor=False)

    ax[1].grid(axis='both')
    fig.tight_layout()
    plt.show()

In [ ]:
plt.plot(np.mean(shap_values[0][1], axis=1))
plot_seq(shap_values[0][10])

## Compute Shap values using Gradient Explainer

In [ ]:
lstm = stress_lstm

In [ ]:
# create GradientExplainer from model and training set
explainer = shap.GradientExplainer(lstm, x_train)

In [ ]:
shap_values = explainer.shap_values(x_test, nsamples=1000)
shap_matrix = shap_values[0]
shap_matrix.shape

In [ ]:
#np.save("./model_logs/best_stress_LSTM/shap_values.npy", shap_matrix)

In [ ]:
# average SHAP values across 21 sequence periods, then across 1000 samples
# results in 37 feature SHAP values from 21000 points
shap_per_sample = np.mean(shap_matrix, axis=1)
print("mean across sequences:", shap_per_sample.shape)
shap_std_per_feature = np.std(shap_per_sample, axis=0)
shap_per_feature = np.mean(shap_per_sample, axis=0)
print("mean across samples:", shap_per_feature.shape)
# plot the variance of shap values
#plt.errorbar(range(len(shap_per_feature)), shap_per_feature, yerr=shap_std_per_feature)

# Stats

## Create SHAP df

In [ ]:
stress_shap = np.load("./model_logs/best_stress_LSTM/shap_values.npy")
#happy_shap = np.load("./model_logs/best_happy_LSTM/shap_values.npy")

In [ ]:
shap_matrix = stress_shap
shap_per_sample = np.mean(shap_matrix, axis=1)
feature_per_sample = np.mean(x_test, axis=1)

In [ ]:
shap.summary_plot(shap_per_sample, features=feature_per_sample, feature_names=features_fr, max_display=40, plot_type="bar")

In [ ]:
# computes the actual value used for the plot
shap_mag = np.sum(np.abs(shap_per_sample), axis=0)
feature_shap_df = pd.DataFrame(list(zip(features_fr, shap_mag)), columns=["feature", "shap"])
feature_shap_df = feature_shap_df.set_index("feature")

## Correlations with social

In [ ]:
features_name = {'accel_norm': "Accéléromètre",
 'accel_low_activity_ratio': "Accéléromètre - faible activité",
 'accel_high_activity_ratio': "Accéléromètre - forte activité",
 'accel_total_activity_ratio': "Accéléromètre - activité totale",
 'app_running_count': "Apps - nb en cours",
 'battery_temp': "Batterie - température",
 'battery_volt': "Batterie - voltage",
 'battery_level': "Batterie - niveau",
 'battery_is_plugged': "Batterie - branchée",
 'battery_is_discharging': "Batterie - décharge",
 'bt_mac_addr_count': "Bluetooth - nb d'appareils",
 'call_duration': "Appels - durée",
 'call_phone_hash': "Appels - nb de contacts",
 'call_type_incoming': "Appels - nb entrants",
 'call_type_missed': "Appels - nb manqués",
 'call_type_outgoing': "Appels - nb sortants",
 'loc_accuracy': "Lieu - précision",
 'loc_x': "Lieu - coordonée x",
 'loc_y': "Lieu - coordonée y",
 'loc_pairwise_dist': "Lieu - distance parcourue",
 'sms_phone_hash': "SMS - nb de contacts",
 'sms_type_incoming': "SMS - nb entrants",
 'sms_type_outgoing': "SMS - nb sortants",
 'wifi_scan_total': "Wi-Fi - nb réseaux",
 'wifi_rssi_avg': "Wi-Fi - signal moyen",
 'wifi_rssi_std': "Wi-Fi - signal variance",
 'day_of_month': "Jour du mois",
 'is_weekend': "Weekend",
 'ema_completion_offset': "Date d'autoévaluation",
 'ema_completion_hour': "Heure d'autoévaluation",
 'day_of_week_0.0': "Lundi",
 'day_of_week_1.0': "Mardi",
 'day_of_week_2.0': "Mercredi",
 'day_of_week_3.0': "Jeudi",
 'day_of_week_4.0': "Vendred",
 'day_of_week_5.0': "Samedi",
 'day_of_week_6.0': "Dimanche",
 'happy': "Humeur",
 'stress': "Stress",
 'productive': "Productivité",
 'eat_healthy': "Alimentation saine",
 'sleep_h': "Nb heures sommeil"}

In [ ]:
# load df used to extract sequences
df = pd.read_csv("./generated-datasets/stress/all_users_stress_8H_dl_instances.csv")
# drop rows without label
df = df.loc[df["happy"].notna()]
# drow unwanted columns
features_df = df.drop(columns=["Unnamed: 0", "userid", "timestamp", "social_h", "stress", "happy", "productive", "eat_healthy", "sleep_h"])
# get remaining features name
features_eng = features_df.columns.to_list()

In [ ]:
features_df = features_df.rename(columns=features_name)
features_fr = features_df.columns.to_list()

In [ ]:
# create dictionary of correlations
social_pearsons = {col:stats.pearsonr(df["social_h"].to_numpy(), features_df[col].to_numpy()) for col in features_fr}

In [ ]:
# create df from correlation dictionary and add p-value columns
social_df = pd.DataFrame.from_dict(social_pearsons, orient="index", columns=["pearson_r", "p"])
social_df.index = social_df.index.rename("feature")
#social_df["p<0.05"] = np.where(social_df["p"] < 0.05, True, False)
#social_df["p<0.01"] = np.where(social_df["p"] < 0.01, True, False)
#social_df["p<0.001"] = np.where(social_df["p"] < 0.001, True, False)
#social_df["p<0.0001"] = np.where(social_df["p"] < 0.0001, True, False)

## Merge Pearson and SHAP

In [ ]:
# merge with stats_df
stats_df = social_df.merge(feature_shap_df, left_index=True, right_index=True)
stats_df = stats_df.fillna(0)

In [ ]:
stats_df["shap_rank"] = stats_df['shap'].abs().rank(method='max')
#merged["shap_std_rank"] = merged['shap_std'].abs().rank(method='max')
stats_df["pearson_mag"] = stats_df['pearson_r'].abs()
stats_df["pearson_rank"] = stats_df['pearson_mag'].abs().rank(method='max')

## Kendall Tau statistical test

In [ ]:
stats_df.sort_values(by="shap", ascending=False, key=abs)

In [ ]:
kendalltau = stats.kendalltau(stats_df["shap_rank"].to_numpy(), stats_df["pearson_rank"].to_numpy())
print(kendalltau)
weighted_tau = stats.weightedtau(stats_df["shap"].abs().to_numpy(), stats_df["pearson_mag"].abs().to_numpy(), rank=True, additive=True)
print(weighted_tau)

# happy
# KendalltauResult(correlation=0.04057101069932574, pvalue=0.723968494625439)
# WeightedTauResult(correlation=-0.14459490694812802, pvalue=nan)

# stress
#KendalltauResult(correlation=0.04658153080292956, pvalue=0.6851247178913382)
#WeightedTauResult(correlation=-0.05875980292548424, pvalue=nan)

## Visualization

In [ ]:
#stress_stats = pd.read_csv("./generated-datasets/stress/stats.csv")
happy_stats = pd.read_csv("./generated-datasets/happy/stats.csv")

In [ ]:
stats_df = happy_stats
stats_df = stats_df.rename(columns={"Unnamed: 0":"features"})
stats_df = stats_df.sort_values(by="shap", ascending=False, key=abs)
stats_df.head()

In [ ]:
stats_df = stats_df.sort_values(by="shap", ascending=False, key=abs)
top10 = stats_df.iloc[:10].feature
top10_val = stats_df.iloc[:10]["shap"].abs().to_numpy()

In [ ]:
# Plot top features
plt.rcParams["figure.figsize"] = (4, 10)
plt.rcParams["font.size"] = 16
plt.rcParams["legend.loc"] = "upper left"

ax = plt.subplot()
y = np.arange(10)
ax.barh(y, np.flip(top10_val), height=0.7, align='center')
ax.set_yticks(y)
ax.set_yticklabels(np.flip(top10))
ax.set_xticks([0, 1, 2])
plt.grid(axis="x")

plt.xlabel("Valeur de Shapley")
plt.ylabel("Feature")
plt.title("Humeur - Top 10 Features")
plt.show()

In [ ]:
# Plot ordered SHAP with pearsons r
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 16
plt.rcParams["legend.loc"] = "upper left"

# x y data
y_shap = stats_df["shap"].to_numpy()
y_pearson = stats_df["pearson_mag"].to_numpy()
y_pearson_scaled = np.interp(y_pearson,
                     (-1, 1),
                     (y_shap.min(), y_shap.max())
                     )
x = np.arange(y_shap.shape[0])

# figure
fig = plt.figure()
# subfigure 1
ax1 = plt.subplot(211)
color = "tab:blue"
ax1.set_ylabel("Valeur de Shapley", color=color)
ax1.bar(x, y_shap, width=0.8, align='center', color=color, label="Shapley value")
ax1.tick_params(axis="y", labelcolor=color)
plt.grid(axis="y")
plt.title("Humeur - Importance des features")

#subfigure 2
ax2 = plt.subplot(212)
color = "tab:orange"
ax2.set_xlabel("Feature #")
ax2.set_ylabel("Pearson r", color=color)
ax2.bar(x, y_pearson, width=0.5, align='center', color=color, label="Pearson r")
ax2.tick_params(axis="y", labelcolor=color)
plt.grid(axis="y")
plt.title("Humeur - Corrélation à l'activité sociale")
#alignYaxes([ax1, ax2], align_values=(0,0))

ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
def alignYaxes(axes, align_values=None):
    '''Align the ticks of multiple y axes

    Args:
        axes (list): list of axes objects whose yaxis ticks are to be aligned.
    Keyword Args:
        align_values (None or list/tuple): if not None, should be a list/tuple
            of floats with same length as <axes>. Values in <align_values>
            define where the corresponding axes should be aligned up. E.g.
            [0, 100, -22.5] means the 0 in axes[0], 100 in axes[1] and -22.5
            in axes[2] would be aligned up. If None, align (approximately)
            the lowest ticks in all axes.
    Returns:
        new_ticks (list): a list of new ticks for each axis in <axes>.

        A new sets of ticks are computed for each axis in <axes> but with equal
        length.
    '''
    from matplotlib.pyplot import MaxNLocator

    nax=len(axes)
    ticks=[aii.get_yticks() for aii in axes]
    if align_values is None:
        aligns=[ticks[ii][0] for ii in range(nax)]
    else:
        if len(align_values) != nax:
            raise Exception("Length of <axes> doesn't equal that of <align_values>.")
        aligns=align_values

    bounds=[aii.get_ylim() for aii in axes]

    # align at some points
    ticks_align=[ticks[ii]-aligns[ii] for ii in range(nax)]

    # scale the range to 1-100
    ranges=[tii[-1]-tii[0] for tii in ticks]
    lgs=[-np.log10(rii)+2. for rii in ranges]
    igs=[np.floor(ii) for ii in lgs]
    log_ticks=[ticks_align[ii]*(10.**igs[ii]) for ii in range(nax)]

    # put all axes ticks into a single array, then compute new ticks for all
    comb_ticks=np.concatenate(log_ticks)
    comb_ticks.sort()
    locator=MaxNLocator(nbins='auto', steps=[1, 2, 2.5, 3, 4, 5, 8, 10])
    new_ticks=locator.tick_values(comb_ticks[0], comb_ticks[-1])
    new_ticks=[new_ticks/10.**igs[ii] for ii in range(nax)]
    new_ticks=[new_ticks[ii]+aligns[ii] for ii in range(nax)]

    # find the lower bound
    idx_l=0
    for i in range(len(new_ticks[0])):
        if any([new_ticks[jj][i] > bounds[jj][0] for jj in range(nax)]):
            idx_l=i-1
            break

    # find the upper bound
    idx_r=0
    for i in range(len(new_ticks[0])):
        if all([new_ticks[jj][i] > bounds[jj][1] for jj in range(nax)]):
            idx_r=i
            break

    # trim tick lists by bounds
    new_ticks=[tii[idx_l:idx_r+1] for tii in new_ticks]

    # set ticks for each axis
    for axii, tii in zip(axes, new_ticks):
        axii.set_yticks(tii)

    return new_ticks

## Comparing Stress and Mood

In [ ]:
stress_stats = pd.read_csv("./generated-datasets/stress/stats.csv")
happy_stats = pd.read_csv("./generated-datasets/happy/stats.csv")

In [ ]:
labels_stats = stress_stats.merge(happy_stats, left_on="pearson_rank", right_on="pearson_rank")
labels_stats = labels_stats.sort_values("shap_x", ascending=False, key=abs)

In [ ]:
kendalltau = stats.kendalltau(labels_stats["shap_rank_x"].to_numpy(), labels_stats["shap_rank_y"].to_numpy())
print(kendalltau)
weighted_tau = stats.weightedtau(labels_stats["shap_rank_x"].abs().to_numpy(), labels_stats["shap_rank_y"].abs().to_numpy(), rank=True, additive=True)
print(weighted_tau)

In [ ]:
# Plot ordered SHAP with pearsons r
plt.rcParams["figure.figsize"] = (8, 6)

y_stress = labels_stats["shap_x"].to_numpy()
y_happy = labels_stats["shap_y"].to_numpy()


x = np.arange(y_stress.shape[0])

fig = plt.figure()

ax1 = plt.subplot(211)

color = "tab:blue"
ax1.set_ylabel("Stress", color=color)
ax1.bar(x, y_stress, width=0.8, align='center', color=color, label="Shapley value")
ax1.tick_params(axis="y", labelcolor=color)

ax2 = plt.subplot(212)

ax2.set_xlabel("Feature #")
ax2.set_ylabel("Humeur", color=color)
ax2.bar(x, y_happy, width=0.8, align='center', color=color, label="Pearson r")
ax2.tick_params(axis="y", labelcolor=color)

#alignYaxes([ax1, ax2], align_values=(0,0))

ax.legend()
plt.title("Stress - Feature importance")
fig.tight_layout()
plt.show()